In [19]:
from math import exp
from math import log
import numpy as np  
import pandas as pd





In [18]:
!git add SMR.ipynb
!git commit -m "update"
!git push origin main

On branch main
nothing to commit, working tree clean


To https://github.com/hoanganhdo2316-png/AIO.git
 * [new branch]      main -> main
